In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction import text 
from nltk.corpus import stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import pandas as pd

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ttonny0326/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ttonny0326/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ttonny0326/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ttonny0326/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
news_data = pd.read_csv('/Users/ttonny0326/GitHub_Project/Topic_Modelling/News/Bloom_Berg_News.csv')
news_data.drop_duplicates()

,Title,Sub_Title,Total_title,Author,Date,Content
0,Five Takeaways From the Paris Summit to Fix Gl...,,Five Takeaways From the Paris Summit to Fix Gl...,Akshat Rathi and Natasha White+Follow,2023-06-23T00:00:05.250000Z,After two days of talks overhauling the global...
1,US Economy May Beat the Odds This Year and Dod...,,US Economy May Beat the Odds This Year and Dod...,Ben Holland+Follow,2023-06-23T00:00:05.250000Z,"This is Washington Edition, the newsletter abo..."
2,"Deluge Disrupts Copper Mines, Crops and Transp...",,"Deluge Disrupts Copper Mines, Crops and Transp...",James Attwood+Follow,2023-06-23T00:00:05.250000Z,Central Chile’s first major rain storms this y...
3,Banks Hit by Worst Week Since May as Regulator...,,Banks Hit by Worst Week Since May as Regulator...,Matt Turner+Follow,2023-06-23T00:00:05.250000Z,Bank stocks’ recovery from their swoon earlier...
4,Chinese Builder Halts All Offshore Debt Paymen...,,Chinese Builder Halts All Offshore Debt Paymen...,Jacob Gu+Follow,2023-06-23T00:00:05.250000Z,A Hong Kong-listed Chinese real estate develop...
...,...,...,...,...,...,...
2164,Amazon CEO Asks His Hollywood Studio to Explai...,,Amazon CEO Asks His Hollywood Studio to Explai...,None,2023-07-05T09:00:10.387Z,Amazon.com Inc. CEO Andy Jassy is taking a har...
2165,Zuckerberg Under Fire in China After Report of...,,Zuckerberg Under Fire in China After Report of...,None,2023-07-05T08:38:53.772Z,Mark Zuckerberg is feeling the heat in China —...
2166,Taiwan’s Powerchip Teams Up With SBI to Build ...,,Taiwan’s Powerchip Teams Up With SBI to Build ...,None,2023-07-05T08:29:03.191Z,Taiwan’s Powerchip Semiconductor Manufacturing...
2167,Coldplay Concert Drives Rush to Online Lending...,,Coldplay Concert Drives Rush to Online Lending...,None,2023-07-05T08:07:55.870Z,"A lineup of post-pandemic music concerts, from..."


In [5]:

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator)

nltk_stopwords = set(stopwords.words('english'))
my_stop_words = text.ENGLISH_STOP_WORDS.union(nltk_stopwords)
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Lemmatize the text
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Remove stop words
    tokens = [token for token in tokens if token not in my_stop_words]
    
    # Remove punctuation and non-alphabetic tokens
    tokens = [token for token in tokens if token.isalpha()]
    
    return tokens

# Apply preprocessing
news_data['processed_content'] = news_data['Content'].apply(preprocess_text)


In [6]:
news_data.head()

,Title,Sub_Title,Total_title,Author,Date,Content,processed_content
0,Five Takeaways From the Paris Summit to Fix Gl...,,Five Takeaways From the Paris Summit to Fix Gl...,Akshat Rathi and Natasha White+Follow,2023-06-23T00:00:05.250000Z,After two days of talks overhauling the global...,"[day, talk, overhauling, global, financial, ta..."
1,US Economy May Beat the Odds This Year and Dod...,,US Economy May Beat the Odds This Year and Dod...,Ben Holland+Follow,2023-06-23T00:00:05.250000Z,"This is Washington Edition, the newsletter abo...","[washington, edition, newsletter, money, power..."
2,"Deluge Disrupts Copper Mines, Crops and Transp...",,"Deluge Disrupts Copper Mines, Crops and Transp...",James Attwood+Follow,2023-06-23T00:00:05.250000Z,Central Chile’s first major rain storms this y...,"[central, chile, major, rain, storm, year, led..."
3,Banks Hit by Worst Week Since May as Regulator...,,Banks Hit by Worst Week Since May as Regulator...,Matt Turner+Follow,2023-06-23T00:00:05.250000Z,Bank stocks’ recovery from their swoon earlier...,"[bank, stock, recovery, swoon, earlier, year, ..."
4,Chinese Builder Halts All Offshore Debt Paymen...,,Chinese Builder Halts All Offshore Debt Paymen...,Jacob Gu+Follow,2023-06-23T00:00:05.250000Z,A Hong Kong-listed Chinese real estate develop...,"[hong, chinese, real, estate, developer, said,..."


In [7]:
# Rejoin the processed tokens to create processed text
news_data['processed_text'] = news_data['processed_content'].str.join(' ')

# Initialize CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Fit and transform the processed titles
dtm = vectorizer.fit_transform(news_data['processed_text'])

# Convert to pandas DataFrame for better visual
dtm_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())

# Display the document-term matrix
dtm_df.head()



,aaa,aaron,aashna,ab,abadia,abandon,abandoned,abandoning,abate,abated,...,zoo,zoom,zoopla,zouari,zr,zuckerberg,zuk,zuma,zunyi,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Define the number of topics
n_topics = 10

# Initialize LDA model
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)

# Fit the model to the data
lda_model.fit(dtm)

# For each topic, print the top 15 most representative words
for index, topic in enumerate(lda_model.components_):
    print(f"Top words for Topic #{index}")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print("\n")


Top words for Topic #0
['group', 'sale', 'market', 'fund', 'new', 'million', 'people', 'firm', 'according', 'ha', 'china', 'billion', 'year', 'company', 'said']


Top words for Topic #1
['global', 'bloomberg', 'data', 'million', 'day', 'russia', 'export', 'barrel', 'year', 'crude', 'oil', 'market', 'india', 'said', 'ha']


Top words for Topic #2
['president', 'russian', 'debt', 'deal', 'nation', 'european', 'government', 'year', 'eu', 'country', 'billion', 'russia', 'ha', 'ukraine', 'said']


Top words for Topic #3
['world', 'ai', 'say', 'year', 'think', 'going', 'said', 'people', 'new', 'chinese', 'like', 'know', 'ha', 'wa', 'china']


Top words for Topic #4
['week', 'cost', 'production', 'investment', 'power', 'gas', 'new', 'government', 'energy', 'uk', 'wa', 'company', 'year', 'ha', 'said']


Top words for Topic #5
['country', 'prigozhin', 'bank', 'court', 'law', 'group', 'twitter', 'party', 'government', 'putin', 'wagner', 'new', 'wa', 'ha', 'said']


Top words for Topic #6
['bloom

In [8]:
# Transform the data
lda_transformed = lda_model.transform(dtm)

# Create a DataFrame
topic_df = pd.DataFrame(lda_transformed, columns=[f"Topic_{i}" for i in range(n_topics)])

# Add a column with the dominant topic per document
topic_df["dominant_topic"] = topic_df.idxmax(axis=1)

# Add the dominant topic data to the original news data
news_data = pd.concat([news_data, topic_df["dominant_topic"]], axis=1)

# Display the first few rows of the updated news data
news_data.head()


,Title,Sub_Title,Total_title,Author,Posted_Time,Content,processed_content,processed_text,dominant_topic
0,Xi Urges Open Supply Chains After Curb on Key ...,,Xi Urges Open Supply Chains After Curb on Key ...,Bloomberg News,2023/7/5T00:00:05.250Z,Chinese leader Xi Jinping called on nations to...,"[chinese, leader, xi, jinping, called, nation,...",chinese leader xi jinping called nation spurn ...,Topic_6
1,Xi Urges Open Supply Chains After Curb on Key ...,,Xi Urges Open Supply Chains After Curb on Key ...,Bloomberg News,2023/7/5T00:00:05.250Z,Chinese leader Xi Jinping called on nations to...,"[chinese, leader, xi, jinping, called, nation,...",chinese leader xi jinping called nation spurn ...,Topic_6
2,China’s Chip-Metals Producers Surge on Export ...,,China’s Chip-Metals Producers Surge on Export ...,Bloomberg News,2023/7/4T00:00:05.250Z,The shares of Chinese metals producers surged ...,"[share, chinese, metal, producer, surged, beij...",share chinese metal producer surged beijing im...,Topic_6
3,What Are Gallium and Germanium? Niche Metals H...,,What Are Gallium and Germanium? Niche Metals H...,Archie Hunter+Follow,2023/7/3T00:00:05.250Z,China is clamping down on exports of two obscu...,"[china, clamping, export, obscure, crucial, me...",china clamping export obscure crucial metal es...,Topic_6
4,Xi’s Metal Curbs Risk Backfiring as G-7 Seeks ...,,Xi’s Metal Curbs Risk Backfiring as G-7 Seeks ...,Bloomberg News,2023/7/4T00:00:05.250Z,China’s decision to control the export of two ...,"[china, decision, control, export, key, metal,...",china decision control export key metal showed...,Topic_6


In [9]:
news_data.to_excel('/Users/ttonny0326/GitHub_Project/Topic_Modelling/Result/LDA_bloomberg_result.xlsx', index=False)

In [9]:
# Define function to get top words for each topic
def get_top_words(lda_model, vectorizer, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(lda_model.components_):
        top_words[f"Topic_{topic_idx}"] = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    return pd.DataFrame(top_words)

# Get top 15 words for each topic
n_top_words = 15
top_words_df = get_top_words(lda_model, vectorizer, n_top_words)

# Display the DataFrame
print(top_words_df)


      Topic_0    Topic_1     Topic_2  Topic_3     Topic_4     Topic_5  \
0        said         ha        said    china        said        said   
1     company       said     ukraine       wa          ha          ha   
2        year      india          ha       ha        year          wa   
3     billion     market      russia     know     company         new   
4       china        oil     billion     like          wa      wagner   
5          ha      crude     country  chinese          uk       putin   
6   according       year          eu      new      energy  government   
7        firm     barrel        year   people  government       party   
8      people     export  government     said         new     twitter   
9     million     russia    european    going         gas       group   
10        new        day      nation    think       power         law   
11       fund    million        deal     year  investment       court   
12     market       data        debt      say  prod

In [25]:
top_words_df.to_excel('/Users/ttonny0326/GitHub_Project/Topic_Modelling/Result/LDATop_Words_bloomberg.xlsx', index=False)